In [1]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/anvarnarz/praktikum_datasets/main/housing_data_08-02-2021.csv')
df.head() 

,location,district,rooms,size,level,max_levels,price
0,"город Ташкент, Юнусабадский район, Юнусабад 8-...",Юнусабадский,3,57,4,4,52000
1,"город Ташкент, Яккасарайский район, 1-й тупик ...",Яккасарайский,2,52,4,5,56000
2,"город Ташкент, Чиланзарский район, Чиланзар 2-...",Чиланзарский,2,42,4,4,37000
3,"город Ташкент, Чиланзарский район, Чиланзар 9-...",Чиланзарский,3,65,1,4,49500
4,"город Ташкент, Чиланзарский район, площадь Актепа",Чиланзарский,3,70,3,5,55000


In [2]:
import numpy as np
df['price'].replace('Договорная',np.nan,inplace=True)
df['size'].replace('Площадьземли:1сот',100,inplace=True)
df['price']=df['price'].astype(float)
df['size']=df['size'].astype(float)


C:\Users\user\AppData\Local\Temp\ipykernel_8620\2976548401.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['price'].replace('Договорная',np.nan,inplace=True)
C:\Users\user\AppData\Local\Temp\ipykernel_8620\2976548401.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For ex

In [3]:
# NAN QIYMATLARNI ALOHIDA AJRATIB OLIB DATAFRAMEDAN O'CHIRIB TASHLADIK KEYIN ISHLATAMIZ
df_missing_price = df[df['price'].isna()]
df = df.dropna(subset=['price'])
df = df[df['price'] < 10_000_000]
df = df[df['price'] >= 1000]


In [4]:
df['kocha'] = df['location'].str.extract(r'район, (.+)')

In [5]:
df = df.drop(columns=['location'])

In [6]:
df.kocha.value_counts()#.head(700)

kocha
Чиланзар                    112
Кушбеги                     110
Нукус                        91
Буюк Ипак Йули               88
Бабура                       82
                           ... 
Улица феруза                  1
Ракат махалля                 1
м-в Феруза                    1
ГОСПИТАЛЬНЫЙ(банковская)      1
Базар Кадышева                1
Name: count, Length: 1502, dtype: int64

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7463 entries, 0 to 7564
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   district    7463 non-null   object 
 1   rooms       7463 non-null   int64  
 2   size        7463 non-null   float64
 3   level       7463 non-null   int64  
 4   max_levels  7463 non-null   int64  
 5   price       7463 non-null   float64
 6   kocha       7433 non-null   object 
dtypes: float64(2), int64(3), object(2)
memory usage: 466.4+ KB


In [8]:
df.loc[df[df['kocha'].isna()].index,'kocha']=df.loc[df[df['kocha'].isna()].index,'district']

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7463 entries, 0 to 7564
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   district    7463 non-null   object 
 1   rooms       7463 non-null   int64  
 2   size        7463 non-null   float64
 3   level       7463 non-null   int64  
 4   max_levels  7463 non-null   int64  
 5   price       7463 non-null   float64
 6   kocha       7463 non-null   object 
dtypes: float64(2), int64(3), object(2)
memory usage: 724.5+ KB


In [10]:
kocha_counts = df['kocha'].value_counts()
df['kocha_grouped'] = df['kocha'].replace(kocha_counts[kocha_counts <= 2].index, 'Other')
# "Other" guruhini 5 ta kichik guruhga bo‘lish
df.loc[df['kocha_grouped'] == 'Other', 'kocha_grouped'] = pd.qcut(
    df.loc[df['kocha_grouped'] == 'Other', 'price'],
    q=5, 
    labels=['other_very_low', 'other_low', 'other_medium', 'other_high', 'other_very_high']
)



In [11]:
# "other_very_high" bo'lgan eng katta 20 ta qiymatni aniqlaymiz
top_20_other_high = df[df['kocha_grouped'] == 'other_very_high'].nlargest(20, 'price')

# Ushbu qiymatlarni asl "kocha" nomlari bilan almashtiramiz
df.loc[top_20_other_high.index, 'kocha_grouped'] = df.loc[top_20_other_high.index, 'kocha']

In [12]:
df = df.drop(columns=['kocha'])

In [13]:
df

,district,rooms,size,level,max_levels,price,kocha_grouped
0,Юнусабадский,3,57.0,4,4,52000.0,Юнусабад 8-й квартал
1,Яккасарайский,2,52.0,4,5,56000.0,other_high
2,Чиланзарский,2,42.0,4,4,37000.0,Чиланзар 2-й квартал
3,Чиланзарский,3,65.0,1,4,49500.0,Чиланзар 9-й квартал
4,Чиланзарский,3,70.0,3,5,55000.0,площадь Актепа
...,...,...,...,...,...,...,...
7560,Яшнободский,1,38.0,5,5,24500.0,Городок Авиастроителей 2-й квартал
7561,Яшнободский,2,49.0,1,4,32000.0,1-й проезд Авиасозлар
7562,Шайхантахурский,2,64.0,3,9,40000.0,Зульфияханум
7563,Мирзо-Улугбекский,1,18.0,1,4,11000.0,Буюк Ипак Йули


In [14]:
from sklearn.model_selection import StratifiedShuffleSplit
strafied_splits=StratifiedShuffleSplit(n_splits=1,test_size=0.2,random_state=42)

In [15]:
for train_ix,test_ix in strafied_splits.split(df,df['district']):
    strat_train_set=df.iloc[train_ix]
    strat_test_set=df.iloc[test_ix]

In [16]:
strat_train_set

,district,rooms,size,level,max_levels,price,kocha_grouped
665,Учтепинский,3,70.0,3,5,59000.0,Чиланзар 31-й квартал
3680,Юнусабадский,1,36.0,3,4,20500.0,other_very_low
7321,Яшнободский,5,92.0,7,9,62000.0,Тузель 3-й квартал
4710,Мирзо-Улугбекский,5,150.0,1,2,97000.0,other_very_high
1235,Чиланзарский,2,52.0,1,4,39500.0,Чиланзар 5-й квартал
...,...,...,...,...,...,...,...
2248,Мирзо-Улугбекский,2,55.0,5,5,33000.0,3-й проезд Шуртепа
4291,Яшнободский,1,28.0,1,3,19000.0,Чулпон
116,Юнусабадский,2,70.0,4,9,60000.0,Чинабад
1965,Мирзо-Улугбекский,3,85.0,4,4,84000.0,Буюк Ипак Йули


In [17]:
strat_test_set

,district,rooms,size,level,max_levels,price,kocha_grouped
5871,Яшнободский,3,75.41,10,10,35000.0,Корасув (Лисунова)
7376,Мирзо-Улугбекский,1,35.00,3,4,26000.0,1-й проезд Сайрам
2900,Мирабадский,2,45.00,2,9,37000.0,Массив Госпитальный
6292,Чиланзарский,2,50.00,2,4,40000.0,Чиланзар 6-й квартал
52,Чиланзарский,2,45.00,2,4,48000.0,Чиланзар 2-й квартал
...,...,...,...,...,...,...,...
317,Мирзо-Улугбекский,3,68.00,4,4,24000.0,other_very_low
1612,Чиланзарский,3,63.00,3,4,42000.0,1-й проезд Астрабад
2296,Мирабадский,3,80.00,3,5,72000.0,жилой комплекс Эльбек
1749,Мирзо-Улугбекский,2,65.00,4,4,33000.0,other_low


In [18]:
X=strat_train_set.drop('price',axis=1)
y=strat_train_set['price'].copy()
X2=X.copy()

In [19]:
y_log = np.log1p(y)  # log(1 + price)


In [20]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, OrdinalEncoder, OneHotEncoder

# Raqamli ustunlar
num_cols = ['rooms', 'size', 'level', 'max_levels']

# Kategoriyali ustunlar
ordinal_col = ['kocha_grouped']   # tartibli
onehot_col = ['district']         # nominal

full_pipeline=ColumnTransformer([
 ('num',MinMaxScaler(),num_cols),
 ('cat_kocha',OrdinalEncoder(),ordinal_col),
 ('cat_district',OneHotEncoder(),onehot_col)   
])

X_prepared=full_pipeline.fit_transform(X)

In [21]:
X_prepared[:10]

array([[2.22222222e-01, 9.85728368e-04, 1.11111111e-01, 1.66666667e-01,
        3.22000000e+02, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [0.00000000e+00, 5.00007143e-04, 1.11111111e-01, 1.25000000e-01,
        3.90000000e+01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [4.44444444e-01, 1.30001857e-03, 3.33333333e-01, 3.33333333e-01,
        2.63000000e+02, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        1.00000000e+00],
       [4.44444444e-01, 2.12860184e-03, 0.00000000e+00, 4.166

In [22]:
from sklearn.linear_model import LinearRegression
LR_model=LinearRegression()

LR_model.fit(X_prepared,y)

LinearRegression()

In [23]:
test_data=X.sample(10)
test_label=y.loc[test_data.index]



test_data_prepared=full_pipeline.transform(test_data)



y_predicted=LR_model.predict(test_data_prepared)



In [24]:
pd.DataFrame({'Asl':test_label,'Bashorat':y_predicted})

,Asl,Bashorat
2081,77000.0,50506.699013
7413,30000.0,35653.200721
1694,280000.0,111313.507811
5938,33500.0,46740.015640
1908,31000.0,50850.063663
383,105000.0,123505.740330
6329,53000.0,80835.674644
4900,72000.0,80102.214087
190,36000.0,36673.967768
1701,285000.0,157599.947130


In [25]:
from sklearn.ensemble import RandomForestRegressor
RF_model=RandomForestRegressor()


RF_model.fit(X_prepared,y,)

RandomForestRegressor()

In [26]:
test_data=X.sample(10)
test_label=y.loc[test_data.index]



test_data_prepared=full_pipeline.transform(test_data)



y_predicted=RF_model.predict(test_data_prepared)



In [27]:
pd.DataFrame({'Asl':test_label,'Bashorat':y_predicted})

,Asl,Bashorat
7061,34000.0,37244.890000
1272,34000.0,37336.500000
5586,130000.0,119067.100000
5436,68000.0,63247.857143
2244,39000.0,39235.000000
653,29996.0,29416.145810
4787,77999.0,99224.250000
1614,75000.0,70985.000000
6203,118000.0,116202.800000
7016,87000.0,96920.000000


In [28]:
# LINEARREGRESSION UCHUN
from sklearn.model_selection import cross_val_score

mse_scores=cross_val_score(LinearRegression(),X_prepared,y_log,scoring='neg_mean_squared_error',cv=5)

def display_scores(scores):
    print('score:',scores)
    print('Mean:',scores.mean())
    print('std.dev:',scores.std())



display_scores(np.sqrt(-mse_scores))

score: [0.37393854 0.34018393 0.38967226 0.36204537 0.37551131]
Mean: 0.3682702828597913
std.dev: 0.016554149152066076


In [29]:
# RANDOM FOREST UCHUN
from sklearn.model_selection import cross_val_score

mse_scores=cross_val_score(RandomForestRegressor(random_state=42),X_prepared,y_log,scoring='neg_mean_squared_error',cv=5)

def display_scores(scores):
    print('score:',scores)
    print('Mean:',scores.mean())
    print('std.dev:',scores.std())



display_scores(np.sqrt(-mse_scores))

score: [0.30591452 0.213271   0.27682686 0.25938028 0.28229959]
Mean: 0.26753845170743445
std.dev: 0.030946406028354172


In [30]:
df

,district,rooms,size,level,max_levels,price,kocha_grouped
0,Юнусабадский,3,57.0,4,4,52000.0,Юнусабад 8-й квартал
1,Яккасарайский,2,52.0,4,5,56000.0,other_high
2,Чиланзарский,2,42.0,4,4,37000.0,Чиланзар 2-й квартал
3,Чиланзарский,3,65.0,1,4,49500.0,Чиланзар 9-й квартал
4,Чиланзарский,3,70.0,3,5,55000.0,площадь Актепа
...,...,...,...,...,...,...,...
7560,Яшнободский,1,38.0,5,5,24500.0,Городок Авиастроителей 2-й квартал
7561,Яшнободский,2,49.0,1,4,32000.0,1-й проезд Авиасозлар
7562,Шайхантахурский,2,64.0,3,9,40000.0,Зульфияханум
7563,Мирзо-Улугбекский,1,18.0,1,4,11000.0,Буюк Ипак Йули


In [31]:
from sklearn.ensemble import GradientBoostingRegressor
gbr_model = GradientBoostingRegressor(random_state=42)
scores = cross_val_score(gbr_model, X_prepared, y, scoring='neg_mean_squared_error', cv=5)
display_scores(np.sqrt(-scores))


score: [ 85374.18967331  20787.519043    64405.33415337 160494.81301874
  29885.10521345]
Mean: 72189.39222037731
std.dev: 49928.11442774387


In [32]:
mean_prices=strat_train_set.groupby('kocha_grouped')['price'].mean()
district_mean_price = strat_train_set.groupby('district')['price'].mean()

X2['kocha_grouped_encoded']=X['kocha_grouped'].map(mean_prices)

In [33]:
X2_test=strat_test_set.drop('price',axis=1).copy()
X2_test['kocha_grouped_encoded']=X2_test['kocha_grouped'].map(mean_prices)

In [34]:
# Test setdagi kocha_grouped ustunini train'dan chiqqan o'rtacha narxlarga moslashtiramiz
X2_test = strat_test_set.drop('price', axis=1).copy()
X2_test['kocha_grouped_encoded'] = X2_test['kocha_grouped'].map(mean_prices)

# Agar test setda train'da yo'q bo'lgan kochalar bo'lsa:
X2_test['kocha_grouped_encoded'].fillna(mean_prices.mean(), inplace=True)


C:\Users\user\AppData\Local\Temp\ipykernel_8620\3299872305.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X2_test['kocha_grouped_encoded'].fillna(mean_prices.mean(), inplace=True)


In [35]:

# Raqamli ustunlar
num_cols2 = ['rooms', 'size', 'level', 'max_levels','kocha_grouped_encoded']

# Kategoriyali ustunlar
onehot_cols2 = ['district']         # nominal

full_pipeline=ColumnTransformer([
 ('num',MinMaxScaler(),num_cols2),
 ('cat_district',OneHotEncoder(),onehot_cols2)   
])

X2_prepared=full_pipeline.fit_transform(X2)

In [36]:
# LINEARREGRESSION UCHUN
from sklearn.model_selection import cross_val_score

mse_scores=cross_val_score(LinearRegression(),X2_prepared,y_log,scoring='neg_mean_squared_error',cv=5)

def display_scores(scores):
    print('score:',scores)
    print('Mean:',scores.mean())
    print('std.dev:',scores.std())



display_scores(np.sqrt(-mse_scores))

score: [0.33060662 0.28848283 0.34791873 0.30452932 0.32639237]
Mean: 0.3195859736748659
std.dev: 0.020806900085113367


In [37]:
# LINEARREGRESSION UCHUN
from sklearn.model_selection import cross_val_score

mse_scores=cross_val_score(RandomForestRegressor(random_state=42),X_prepared,y_log,scoring='neg_mean_squared_error',cv=5)

def display_scores(scores):
    print('score:',scores)
    print('Mean:',scores.mean())
    print('std.dev:',scores.std())



display_scores(np.sqrt(-mse_scores))

score: [0.30591452 0.213271   0.27682686 0.25938028 0.28229959]
Mean: 0.26753845170743445
std.dev: 0.030946406028354172


In [38]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'max_features': [ 'sqrt','log2'],
    'min_samples_split': [2, 5]
}

grid_search = GridSearchCV(RandomForestRegressor(random_state=42),
                           param_grid,
                           cv=5,
                           scoring='neg_mean_squared_error',
                           n_jobs=-1,
                           verbose=2)



In [39]:
# LINEARREGRESSION UCHUN
from sklearn.model_selection import cross_val_score

mse_scores=cross_val_score(grid_search,X_prepared,y_log,scoring='neg_mean_squared_error',cv=5)

def display_scores(scores):
    print('score:',scores)
    print('Mean:',scores.mean())
    print('std.dev:',scores.std())



display_scores(np.sqrt(-mse_scores))

Fitting 5 folds for each of 24 candidates, totalling 120 fits
Fitting 5 folds for each of 24 candidates, totalling 120 fits
Fitting 5 folds for each of 24 candidates, totalling 120 fits
Fitting 5 folds for each of 24 candidates, totalling 120 fits
Fitting 5 folds for each of 24 candidates, totalling 120 fits
score: [0.27904385 0.20744579 0.26455001 0.2460929  0.26809879]
Mean: 0.25304626877046016
std.dev: 0.02514971945749524


In [40]:
# Mavjud X2 DataFrame-ga yangi ustunlar qo‘shamiz
X2['room_size_avg'] = X2['size'] / X2['rooms']
X2['is_new_building'] = ((X2['level'] == 1) & (X2['max_levels'] <= 3)).astype(int)
X2['is_top_floor'] = (X2['level'] == X2['max_levels']).astype(int)

# district_encoded_mean_price

X2['district_encoded_mean_price'] = X2['district'].map(district_mean_price)


In [41]:
# Yangi raqamli ustunlar ro'yxatiga qo‘shamiz
num_cols2 = [
    'rooms', 'size', 'level', 'max_levels',
    'kocha_grouped_encoded', 'room_size_avg',
    'district_encoded_mean_price'
]

# Boolean ustunlar — MinMaxScaler uchun yaroqli
bool_cols = ['is_new_building', 'is_top_floor']

# To‘liq raqamli ustunlar to‘plami
all_num_cols = num_cols2 + bool_cols

# Kategorik ustunlar o‘sha-o‘sha qoladi
onehot_cols2 = ['district']


In [42]:
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

full_pipeline = ColumnTransformer([
    ('num', MinMaxScaler(), all_num_cols),
    ('cat_district', OneHotEncoder(), onehot_cols2)
])


In [43]:
X2_prepared = full_pipeline.fit_transform(X2)

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
import numpy as np

mse_scores = cross_val_score(LinearRegression(), X2_prepared, y_log, scoring='neg_mean_squared_error', cv=5)

def display_scores(scores):
    print("score:", scores)
    print("Mean:", scores.mean())
    print("std.dev:", scores.std())

display_scores(np.sqrt(-mse_scores))  # RMSE


score: [0.32878228 0.28557025 0.34791517 0.30139061 0.32504036]
Mean: 0.3177397332752409
std.dev: 0.021853893543437395


In [44]:
X2_prepared = full_pipeline.fit_transform(X2)

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
import numpy as np

mse_scores = cross_val_score(grid_search, X2_prepared, y_log, scoring='neg_mean_squared_error', cv=5)

def display_scores(scores):
    print("score:", scores)
    print("Mean:", scores.mean())
    print("std.dev:", scores.std())

display_scores(np.sqrt(-mse_scores))  # RMSE


Fitting 5 folds for each of 24 candidates, totalling 120 fits


Fitting 5 folds for each of 24 candidates, totalling 120 fits
Fitting 5 folds for each of 24 candidates, totalling 120 fits
Fitting 5 folds for each of 24 candidates, totalling 120 fits
Fitting 5 folds for each of 24 candidates, totalling 120 fits
score: [0.25404262 0.19278866 0.23639819 0.22986248 0.2407133 ]
Mean: 0.23076104861068975
std.dev: 0.020569714520095914


In [ ]:
from sklearn import svm

classifier=svm.SVR(kernel='linear')


mse_scores = cross_val_score(classifier, X2_prepared, y_log, scoring='neg_mean_squared_error', cv=5)

def display_scores(scores):
    print("score:", scores)
    print("Mean:", scores.mean())
    print("std.dev:", scores.std())

display_scores(np.sqrt(-mse_scores))  # RMSE


score: [0.32925989 0.2851975  0.34717035 0.30143635 0.32124974]
Mean: 0.3168627681308448
std.dev: 0.021596977021553276
